<div style="text-align:center;">
    <img src="http://www.cs.wm.edu/~rml/images/wm_horizontal_single_line_full_color.png">
    <h1>CSCI 312, Fall 2025</h1>
    <h1>Effective C, Chapter 4
    <h1>Expressions and operators</h1>
</div>

# Contents

* [Assignments: lvalues and rvalues](#Assignments:-lvalues-and-rvalues)
* [Side effects](#Side-effects)
* [The unary increment and decrement](#The-unary-increment-and-decrement)
* [Bitwise operators](#Bitwise-operators)
    * [Bitwise `AND`, `OR`, `XOR`, and `NOT`](#Bitwise-AND,-OR,-XOR,-and-NOT)
        * [Bitwise `AND`](#Bitwise-AND)
        * [Bitwise `OR`](#Bitwise-OR)
        * [Bitwise `XOR`](#Bitwise-XOR)
        * [Bitwise `NOT`](#Bitwise-NOT)
    * [Bitwise shifts](#Bitwise-shifts)
        * [¡Cuidado! ¡llamas! 🦙🦙 🐞🐞](#¡Cuidado!--¡llamas!-🦙🦙-🐞🐞)
    * [Further reading](#Further-reading)
* [Booleans](#Booleans)
    * [Boolean operators](#Boolean-operators)
    * [Comparison operators](#Comparison-operators)
    * [Boolean expressions](#Boolean-expressions)
        * [Don't use `a <= x <= b`](#Don't-use-a-<=-x-<=-b)

# Python vs C vs C++

|           | Python          | C            | C++   | Java |
| :--------:| :-------------: | :----------: | :---: | :---: |
| bitwise `AND` | same as C   | `&`   | same as C   | same as C   |
| bitwise inclusive `OR` | same as C |   `\|`   | same as C   | same as C  |
| bitwise exclusive ```OR``` | same as C   | `^`   | same as C | same as C |
| one's complement | same as C   | `~`   | same as C | same as C   |
| left shift | same as C | `<<` | same as C | same as C |
| right shift | same as C | `>>` | same as C | same as C |
| booleans: | `True, False` | `true, false` | same as C | same as C |
| logical AND:      | `and` | `&&` | `&&` or `and` | same as C |
| logical OR:      | `or` | <code>&#124;&#124;</code> | <code>&#124;&#124;</code> or `or` | same as C |
| logical NOT:      | `not` | `!` | same as C | same as C |
| comparison operators: | same as C | `<, <=, ==, !=, >=, >` | same<sup>1</sup> | same as C |
| identity | `is` |  |
| membership | `in` |  |
| conditional expression | `expr1 if expr2 else expr3`  | `expr1 ? expr2 : expr3` | same as C | same as C|

<sup>1</sup> C++20 added a [three-way comparison operator](https://en.cppreference.com/w/cpp/language/operator_comparison) `<=>`, also known as "the spaceship".  Amusingly, it acts like Fortran's arithmetic `if` that the Fortran standards committee has been trying to get rid of since 1990.

# Assignments: lvalues and rvalues

Now is a good time to introduce the concepts of **lvalues** and **rvalues**.

Roughly speaking, an **lvalue** is something that can be assigned to (and thus appear on the left-hand side of an assignment, while an **rvalue** is a value that can be assigned (and thus appear on the right-hand side of an assignment). 

In particular, temporary variables can be rvalues but not lvalues, as in the following code snippet:

```
int a = 42;
int b = 54;
int c;

c = a + b;
```

Here `a + b` is an rvalue, while `a,b,c` are lvalues. It makes no sense to try to use the temporary `a + b` as an rvalue:

```
a + b = 6;  // Wut?
```

We will see **rvalue semantics** show up in a substantive way in C++ classes.

# Side effects

A **side effect** is a change to the state of the execution environment, e.g., 
* i/o,
* assignment,
* calling a function that does any of these things.

Side effects are most commonly discussed in connection with function calls.

For instance, some authors suggest applying a random permutation before applying quicksort.  This has the side effect of changing the pseudo-random generator.

# The unary increment and decrement

C/C++ has two special **unary operators** (i.e., there is only one operand) for increasing and decreasing a variable by 1.  These operators, `++` and `--`, exist because these operations are very common, particularly in `for` loops.

In [ ]:
cat -n src/increment.c

These operators can either precede or follow the variable being incremented or decremented.  Their behavior depends on whether they are prefixes or suffixes:
* ```++n``` (prefix increment) means increase the value of ```n``` by ```1``` and use the resulting value;
* ```n++``` (suffix increment) means use the value of ```n``` and then increase the value of ```n``` by ```1```.

The statements ```--n``` and ```n--``` behave similarly.

These operations are usually part of some other statement.  Study the difference between the prefix and suffix versions closely:

In [ ]:
gcc src/increment.c

In [ ]:
./a.out

Sometimes it does not matter whether you use the prefix or suffix versions, but often it matters bigly:
* ```++n``` ==> **increment ```n``` and then use the value of ```n```**;
* ```n++``` ==> **use the value of ```n``` and then increment ```n```**.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px;"/>🐞🐞Using the wrong version of <code>++</code> and <code>--</code> (prefix vs suffix) is a common source of off-by-one errors (OBOEs). 🐞🐞<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px;"/> 

# Bitwise operators

C/C++ has six operators for bit manipulation, also known as **bit twiddling**.  Python shares the same bitwise operators.

These operators can only be applied to integers, but any flavor of integer is a valid operand.

## Bitwise `AND`, `OR`, `XOR`, and `NOT`

The first three of the following operators take two operands, while the last one takes only a single operand:
<pre>
&amp;      bitwise AND
|      bitwise inclusive OR (OR)
^      bitwise exclusive OR (XOR)
~      one's complement (NOT)
</pre>
These operators are applied bitwise.  The bits are either 0 or 1; the resulting values are

| AND   |       | OR    |       |XOR    |       | NOT   |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
|```1 & 1 = 1``` | | ```1 \| 1 = 1``` | | ```1 ^ 1 = 0``` | | ```~1 = 0``` |
|```1 & 0 = 0``` | | ```1 \| 0 = 1``` | | ```1 ^ 0 = 1``` | | ```~0 = 1``` |
|```0 & 1 = 0``` | | ```0 \| 1 = 1``` | | ```0 ^ 1 = 1``` | |              |
|```0 & 0 = 0``` | | ```0 \| 0 = 1``` | | ```0 ^ 0 = 0``` | |              |

You can remember these values by thinking of `1` as `true` and `0` as `false`.

Let's look at some examples.

Consider the integers 42 and 54, written out as 32 bit integers:
<pre>
42 = 00000000 00000000 00000000 00101010
54 = 00000000 00000000 00000000 00110110
</pre>
We can ignore the leading zeroes for the purposes of applying ```&```, ```|```, and ```^``` since none of these operations will change the leading bits.

In [ ]:
cat -n src/twiddling.c

In [ ]:
gcc -std=c23 src/twiddling.c

In [ ]:
./a.out

### Bitwise AND

First we will take the bitwise AND.  We line up the bits and look at the pairs of corresponding bits.  If both are 1 the result is 1; otherwise it is 0 (e.g., 1 & 0 = false AND false = false).

<pre>
42      = 101010
54      = 110110
42 & 54 = 100010 = 32 + 2 = 34
</pre> 

We can use the bitwise AND to mask (turn off) specific bits.  For instance, the number ```0177``` is octal (base-8) ```177```.  This is decimal 127 (64 + 7&#42;8 + 7), which is binary ```1111111```.

*Aside.* We can also derive the binary expansion as follows.  Octal digits are three bit binary numbers.  In this case,
<pre>
1 =   1
7 = 111
7 = 111
</pre>
so putting these bits together we obtain 
<pre>
octal 177 = binary 1 111 111
</pre>

The statement
<code>
    n = n & 01777
</code>
sets all but the low-order 7 bits of ```n```to zero because the value ```0177``` is padded on the left with enough zeros to perform the bitwise AND:
<pre>
2 * 2147483647u          = 11111111 11111111 11111111 11111110
0177                     = 00000000 00000000 00000000 01111110
(2 * 2147483647u) & 0177 = 00000000 00000000 00000000 01111110 = 126
</pre>

We have to use `2147483647u` rather than `2147483647` since the latter is a signed integer and `2 * 2147483647` will overflow:

`printf("%u\n", ((2 * 2147483647u)) & 0177);`

vs.

`printf("%d\n", 2 * 2147483647);`

### Bitwise OR

In the bitwise OR w again line up the bits and look at the pairs of corresponding bits.  If one or both are 1 (e.g., 1 | 0 = true OR false = true) the result is 1; otherwise it is 0 (0 | 0 = false OR false = false).

<pre>
42      = 101010
54      = 110110
42 | 54 = 111110 = 32 + 16 + 8 + 4 + 2 = 62
</pre>

### Bitwise XOR

In the bitwise XOR, if exactly one bit in a pair is 1 the result is 1; otherwise it is 0.

<pre>
42      = 101010
54      = 110110
42 ^ 54 = 011100 = 16 + 8 + 4 = 28
</pre> 

The exclusive OR is used extensively in cryptography since it is easily invertible:

```
printf("%u\n", 42 ^ (42 ^ 54));  // a ^ (a ^ b) = b
printf("%u\n", 54 ^ (42 ^ 54));  // b ^ (a ^ b) = a
```

### Bitwise `NOT`

Applying the one's complement ```~``` turns 0 bits to 1 and 1 bits to 0.  Since this operation will change zero bits we need to take the leading bits into account.
<pre>
 42 = 00000000 00000000 00000000 00101010
~42 = 11111111 11111111 11111111 11010101
    = 256 * (2**24 - 1) + 128 + 64 + 16 + 4 + 1
    = 4294967253
</pre>

## Bitwise shifts

The shift operators shift bits left or right.
<pre>
&lt;&lt;   left shift
&gt;&gt;   right shift
</pre>
These operators take two operands; the left operand is the number to be shifted and the right operand is the number of bit positions to shift.  Thus <code>n &lt;&lt; 2</code> shifts the bits in <code>n</code> two bits to the left.  Zero bits take the place of vacated bits.  **Bits that are shifted past either end disappear.**

Let's look at the effect of left and right shifts, first using base-10 numbers.  Here we shift one decimal digit to the left:
<pre>
    12
   120
  1200
 12000
120000
</pre>
This has the effect of multiplying by 10 with each shift.

Now let's shift right:
<pre>
120000
 12000
  1200
   120
    12
</pre>
This has the effect of dividing by 10 with each shift.  More generally, this is integer division by 10, which throws away any remainder:
<pre>
121111
 12111
  1211
   121
    12
</pre>
Shifting by $k$ decimal digits makes a factor of $10^{k}$ change.

The same holds for base-2:
* a one-bit left shift is the same as multiplication by 2, and
* a one-bit right shift is the same as integer division by 2.

Left shift:
<pre>
    1 =  1
   10 =  2
  100 =  4
 1000 =  8
10000 = 16
</pre>

Right shift:
<pre>
10011 = 19
 1001 =  9
  100 =  4
   10 =  2
    1 =  1
</pre>

Shifting by $k$ bits makes a factor of $2^{k}$ change in the operand.

Notice that eventually we shift all the on bits off the left end of the integer and obtain 0.

Similarly, if we right shift enough times we will shift all the on bits off the right end:

## &iexcl;Cuidado!  &iexcl;llamas! 🦙🦙 🐞🐞

You need to be extra careful if you apply the bitwise shift operators to signed integers.  From Section 6.5.8, Bitwise shift operators, in the C23 standard (emphasis mine):
<blockquote>
4 The result of E1 &lt;&lt; E2 is E1 left-shifted E2 bit positions; vacated bits are filled with zeros. If E1 has an unsigned type, the value of the result is E1 $\times$ $2^{\mathrm{E2}}$, reduced modulo one more than the maximum value representable in the result type.  <b>If E1 has a signed type and nonnegative value, and E1 $\times$ $2^{\mathrm{E2}}$ is representable in the result type, then that is the resulting value; otherwise, **the behavior is undefined**.</b>

5 The result of E1 &gt;&gt; E2 is E1 right-shifted E2 bit positions. If E1 has an unsigned type or if E1 has a signed type and a nonnegative value, the value of the result is the integral part of the quotient of E1 / $2^{\mathrm{E2}}$. <b>If E1 has has a signed type and a negative value, the resulting value is **implementation-defined**.</b>
</blockquote>

Terms like "undefined behavior" or "implementation-dependent behavior" mean **TROUBLE AHEAD**.

<img  src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px;"/> Avoid shifting signed integers.  If you absolutely must, run some experiments to make sure you are certain of the behavior on your system.

## Further reading on bit twiddling

You can find an cornucopia of clever and entertaining bit twiddling hacks [here](https://graphics.stanford.edu/~seander/bithacks.html).

# Booleans

Boolean variables can take on one of two values, ```true``` and ```false```.  Originally C did not have a boolean type.  Instead,
* ```true``` was represented by 1, and
* ```false``` was represented by 0.

These can still be used.

Booleans **finally** became a first-class type in C23.  The type name is `bool`.  They are still printed as integers, however.

In [ ]:
cat -n src/booleans.c

As you can see, [as in Python](https://docs.python.org/3/library/stdtypes.html#truth-value-testing), anything that has a non-zero value converts to ```true```, and anything that has the value ```0``` converts ```false```.

Let's build the executable using the ```clang``` compiler rather than the `gcc` compiler:

In [ ]:
clang -Wall -pedantic src/booleans.c

In [ ]:
./a.out

You should see a warning about an implicit conversion from ```double``` to ```_Bool```.  This is because the conversion is a **narrowing conversion** which takes a variable and converts it to one that is less expressive.

## Boolean operators

C/C++ use the following notation:
* logical AND: ```&&```
* logical OR: ```||```
* logical NOT: ```!```

Alternatively, if you include [the `iso646.h` header file](https://en.cppreference.com/w/cpp/language/operator_alternative.html) you can use words like `and` and `or`.

In [ ]:
cat -n src/and_or_not.c

In [ ]:
gcc -std=c23 -pedantic -Wall src/and_or_not.c

In [ ]:
./a.out

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> Be sure to use <code>&amp;&amp;</code> and <code>||</code> and not accidentally use a single <code>&amp;</code> or <code>|</code>.  The latter are bitwise AND and bitwise OR.  

<a href="https://arstechnica.com/gadgets/2021/07/google-pushed-a-one-character-typo-to-production-bricking-chrome-os-devices/" target="_blank">This article</a> describes how an <code>&amp;</code> instead of <code>&amp;&amp;</code> in an update caused Google to lock users out of their ChromeOS devices.

## Comparison operators

The comparison operators in C/C++ are the same as in Python:
* less than: ```<```
* less than or equal to: ```<=```
* equal to: ```==```
* not equal to: ```!=```
* greater than or equal to: ```>=```
* greater than: ```>```

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"/> C and C++ do not have anything comparable to Python's ```is``` or ```in``` statements.

In [ ]:
cat -n src/comparison.c

In [ ]:
clang -pedantic -Wall src/comparison.c

In [ ]:
./a.out

## Boolean expressions

Boolean expressions in C work the samw way as they do in Python, **except when they don't.**

### Don't use `a <= x <= b`

In Python, ```a <= x <= b``` will evaluate to <code class="kw">True</code> if ```x``` lies in the interval $[a, b]$.

In C, ```a <= x <= b``` will be evaluated as ```(a <= x) <= b```.  Since ```a <= x``` will be either 0 or 1 (false or true), then **if ```b``` is greater than or equal to 1, the statement ```a <= x <= b``` will always evaluate to true!**

In [ ]:
cat -n src/bool_exp.c

In [ ]:
clang -pedantic -Wall src/bool_exp.c

Notice `clang` warns us about the problematic statement at line 8.  You might not always be so fortunate as to get a compiler warning, however, and the statement at line 14 slips by unremarked.

In [ ]:
./a.out

The expression ```a <= x <= b``` is valid C, but likely not what you intended.  If you are lucky, your compiler will warn you about a possible error.

<div class="danger"></div> Don't use <code>a <= x <= b</code> in C.

### Assignment expressions: watch out for `=` vs. `==`

Another error that one can make in C is using ```=``` instead of ```==```:

In [ ]:
cat -n src/assign_exp.c

In [ ]:
clang -pedantic -Wall src/assign_exp.c

In [ ]:
./a.out

In the statement
<pre>
    if (x = 54) {...}
</pre>

at line 8 the value `54` is first assigned to `x`, and then `x` is converted to a boolean.  Since the value is non-zero, the boolean evaluates to ```true```.  **This means the `if` block is always executed!**

In C/C++ and Python this construct is called an **assignment expression**. Assignment expressions also appear in Python starting with Python 3.8 as the [walrus operator](https://docs.python.org/3/reference/expressions.html?highlight=walrus#assignment-expressions) `:=`.  In C, regular assignment `=` has the same effect, increasing the likelihood of errors.

If you are lucky, the compiler will issue a warning, as it does in this case.

This error cannot occur in Python, since Python uses the walrus operator ```:=``` for assignment expressions, and
<pre>
    if x = 54:    
</pre>
is not a valid statement.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> Watch out for <code>=</code> where you mean <code>==</code>.

### Using assignment expressions in C

C hackers **love** to use assignment expressions because used properly they can lead to shorter and clearer code (or shorter and more cryptic code).

For instance, consider the following code to read a file one character at a time and write each character to the screen:

In [ ]:
cat -n src/assign_exp2.c

While functional this code is a bit ugly:
* we call ```getc()``` in two different places,
* reading the first character is a special case, and
* ```c``` starts the loop as the current character being processed but then changes to be the next character.

We can simplify the code using assignment expressions as follows:

In [ ]:
cat -n src/assign_exp3.c

Rewriting the code this way is cleaner:
* the code is shorter,
* there is only one reference to ```getc()```,
* reading the first character is no longer a special case, and
* ```c``` is always the current character being processed.

The logic of the ```while``` loop also reads more clearly:
<blockquote>
Read the next character into <code>c</code> and if it is not the end-of-file print <code>c</code> to the screen.
</blockquote>

In [ ]:
clang -pedantic -Wall src/assign_exp3.c

In [ ]:
./a.out

If opening the file fails, `fopen()` returns the special value `nullptr` (or `NULL`, prior to C23).  We can incorporate a check for this using the same idiom:

In [ ]:
cat -n src/assign_exp4.c

In [ ]:
clang -pedantic -Wall -std=c23 src/assign_exp4.c

In [ ]:
./a.out

## The numerical value of a boolean expression

C/C++ and Python "short-circuit" the evaluation of compound boolean expressions.  Reading from left to right all of these languages stop evaluation as soon as the truth or falseness of an expression can be determined.

For instance, in the compound expression
<code>
    expr1 or expr2 or expr3
</code>
suppose ```expr1``` is false but ```expr2``` is true.  As soon as we encounter the true expression ```expr2``` we know the compound statement is true and ```expr3``` will not be evaluated.

In these situations Python may return a result whose numerical value is not 0 or 1, while C/C++ will always return a result whose numerical value is 0 or 1.

In Python,
<code>
    n = (1 >= 2) or (3.14)
</code>
will result in ```n``` having the value ```3.14``` because for a boolean expression Python returns the first value it encounters that allows it to determine the truth or falseness of the expression.

In C/C++, 
<code>
    n = (1 >= 2) or (3.14)
</code>
will result in ```n``` having the value ```1```.

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> The numerical value of a boolean expression in C/C++ will always be 0 or 1.

# Conditional expressions

C/C++/Java, [like Python](https://docs.python.org/3/reference/expressions.html?highlight=assignment%20expression#conditional-expressions), have conditional expressions.  In C/C++ the conditional expression
<pre>
    m = (a > b) ? a : b;
</pre>
is equivalent to
<pre>
    if (a > b) {
        m = a;
    }
    else {
        m = b;
    }
</pre>

The general syntax is
<pre>
    expr1 ? expr2 : expr3
</pre>

If `expr1` is true, the conditional expression evaluates to `expr2`; otherwise it evaluates to `expr3`.

Here we rewrite `conditional.c` using conditional expressions; the new version is much, much shorter.

In [ ]:
cat -n src/comparison2.c

In [ ]:
clang -pedantic -Wall src/comparison2.c

In [ ]:
./a.out

Let's rewrite the lengthy set of ```if-else``` blocks above in the section on comparison operators:

Here we use a conditional expression to correctly handle the singular vs plural of hour:

In [ ]:
cat -n src/duration.c

In [ ]:
gcc src/duration.c

In [ ]:
./a.out

<img src="https://www.cs.wm.edu/~rml/images/danger.svg" style="height: 30px"> K&amp;R recommend placing the boolean condition in paretheses even though it is not required in order to make it easier to see.  I concur.

# The comma operator

In most situations the C standard does not specify the order of operations.  For instance, in the expression
<pre>
    x = f() + g();
</pre>
it might not be the case that `f()` is evaluated first.  This can lead to problems if the two functions share some common state they both change.

Similarly, the behavior of the expression
<pre>
    i = i + i++;
</pre>
is undefined, since it is not clear when `i` is incremented.

To keep the order of operations under control C/C++ has the notion of **sequence points**.  Per the C standard:
<blockquote>
The presence of a sequence point between the evaluation of expressions A and B implies that every value computation and side effect associated with A is sequenced before every value computation and side effect associated with B. (A summary of the sequence points is given in Annex C.)    
</blockquote>

For instance, the standard specifies a sequence point between the evaluation of the first and second operands of logical `AND`.  The presence of the sequence point makes it possible for `AND` to "short-circuit" and return `false` if the first operand is false.

The comma operator allows us to ensure that one expression is evaluated before the other.  Here is an example to contemplate.

In [ ]:
cat -n src/comma.c

In [ ]:
gcc src/comma.c

In [ ]:
./a.out

# Pointer arithmetic

You can add integers to pointers, and take the difference of pointers.  This can be useful when debugging memory access errors, among other things.

Suppose `p` is pointer to thingies of type `T`, and `k` is an integer.  Then `p + k` is the location in memory that is `k * sizeof(T)` bytes from `p`.

If an `int` is 4 bytes and a `double` is 8 bytes, then
<pre>
    int a[42;
    double x[54]
    a + 3;    // The address that is 3 ints (12 bytes) from the start of a[].
    *(a + 3)  // This is the same as a[3].
    x + 6;    // The address that is 6 doubles (48 bytes) from the start of x[].
</pre>



In [ ]:
cat -n src/pointer.c

In [ ]:
gcc src/pointer.c

In [ ]:
./a.out

The technically not allowed subtraction shows that the array `b` begins 54 `int`s before `a`.  So `b` precedes `a` in memory, even though `a` appears first in the code.  It also shows that there is no space in memory between `a` and `b`.  Thus, when we write to `b[54]` we are writing one `int` past the end of `b`, which is the location of `a[0]`.